In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

C:\Users\ghass\anaconda\envs\LLM\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import torch

In [ ]:
model_name = "stabilityai/stablelm-zephyr-3b"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)


In [ ]:
model_name = "stabilityai/stablelm-zephyr-3b"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)


In [5]:
import pandas as pd
df = pd.read_csv("C:/Users/ghass/Downloads/1000_enron_spam_data.csv")
df

,Unnamed: 0,Message ID,Subject,Message,Spam/Ham,Date
0,0,6568,kirstee ' s role in london,vince :\nthis is precisely the concern i have ...,ham,2000-07-25
1,1,9797,i never sent you this - keep it hush hush ! : ...,online credit breakthroughrepair your credit o...,spam,2002-08-02
2,2,655,copanos changes,- - - - - - - - - - - - - - - - - - - - - - fo...,ham,2000-04-12
3,3,8206,"hrgovcic , hrvoje",please incease the bonus for hrgovcic in vince...,ham,2001-01-13
4,4,25787,a chance to get new logo now,working on your company ' s image ? start with...,spam,2005-06-27
...,...,...,...,...,...,...
995,995,29351,can you help ! ! !,from : mrs . victoria ndlovu\nattn :\nyou may ...,spam,2004-08-13
996,996,32821,dont let age deter you from great sex !,20 minutes is all it takes\nmore info here . ....,spam,2005-05-13
997,997,14476,proposed retention bonuses for netco,"louise ,\ni am sorry to bother you at such a c...",ham,2001-12-28
998,998,21733,epson inkjet cartridges from 5 . 99,"save up to 75 % on inkjet , laser copier suppl...",spam,2005-04-18


In [6]:
df = df[:153]
df

,Unnamed: 0,Message ID,Subject,Message,Spam/Ham,Date
0,0,6568,kirstee ' s role in london,vince :\nthis is precisely the concern i have ...,ham,2000-07-25
1,1,9797,i never sent you this - keep it hush hush ! : ...,online credit breakthroughrepair your credit o...,spam,2002-08-02
2,2,655,copanos changes,- - - - - - - - - - - - - - - - - - - - - - fo...,ham,2000-04-12
3,3,8206,"hrgovcic , hrvoje",please incease the bonus for hrgovcic in vince...,ham,2001-01-13
4,4,25787,a chance to get new logo now,working on your company ' s image ? start with...,spam,2005-06-27
...,...,...,...,...,...,...
148,148,13131,re : louise kitchen - fortune 50 - no . 17,so louise - you are almost as important as opr...,ham,2001-10-02
149,149,21665,make a gift for your girlfriend or wife . adul...,hello !\ni am sure you want to make a gift for...,spam,2005-04-07
150,150,14445,netco memo,see attached draft memo that outlines certain ...,ham,2001-12-21
151,151,4834,v - codin and xa ' nx are cheeap 7 aq,look at this of - fers :\nv - codin - 225 . 00...,spam,2005-03-16


In [14]:
import pprint
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

# Lists to store true labels and predicted labels
true_labels = []
predicted_labels = []
email_class = ""
# Loop through all rows in the DataFrame
for index, row in df.iterrows():
    # Prepare the message
    prompt = (
    "System: Classify the text into Spam or Ham. Please provide me at the end of the response with 'Class of the email:'.\n"
    f"User: {row['Message']} \n"
    "System: Class of the email:?"
    )

    inputs = tokenizer(
    prompt, return_tensors="pt", add_special_tokens=False
    ).to("cuda")

    # generate configuration can be modified to your needs
    tokens = model.generate(
          input_ids=inputs["input_ids"],
          attention_mask=inputs["attention_mask"],
          min_new_tokens=2,
          max_new_tokens=256,
          )[0]

    tokens = tokens[inputs["input_ids"].shape[1]:]
    answer = tokenizer.decode(tokens, skip_special_tokens=True)
    #print(answer)


    # Extract the generated text
    generated_text = answer
    print("example {} : \n".format(index))
    pprint.pprint(generated_text)
    print("\n")

    # Extract the class of the email
    if "Class of the email:" in generated_text:
        # Extract the portion of the text after "Class of the email:"
        email_class = generated_text.split("Class of the email:")[-1].strip().split()[0]  # Take the first word

    pprint.pprint(email_class)
    print("\n")
    #In our prompt if we write it spam or ham in the response it didn't give the class so we used instead "Spam" or "Ham" to recognize it and we formatted it to make it like the true labels
    if email_class == "Spam" or email_class == "Spam." :
        email_class = "spam"
    elif email_class == "Ham" or email_class == "Ham.":
        email_class = "ham"

    # Print the actual Spam/Ham label from the DataFrame for comparison
    actual_class = row["Spam/Ham"]
    pprint.pprint(actual_class)


    # Store the true and predicted labels
    true_labels.append(actual_class)
    predicted_labels.append(email_class)

#153 cuda out of memory

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


example 0 : 

'\nClass of the email: Spam.'


'Spam.'


'ham'


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


example 1 : 

(' Spam or Ham\n'
 '\n'
 'Step 1: Identify the text and look for keywords or phrases that may indicate '
 'whether the email is spam or not.\n'
 '\n'
 'In this email, some keywords and phrases stand out, such as "credit online," '
 '"bad credit," "remove negative credit items," and "click here for more '
 'information." These phrases may indicate that the email is trying to sell a '
 "product or service related to improving one's credit score.\n"
 '\n'
 'Step 2: Determine if the email is spam or not based on the identified '
 'keywords and phrases.\n'
 '\n'
 'Considering the keywords and phrases identified, the email seems to be '
 "promoting a product or service related to improving one's credit score. This "
 'type of email is often considered spam because it may be unsolicited and '
 'sent in bulk to many recipients.\n'
 '\n'
 "Step 3: Provide the 'Class of the email:' at the end of the response.\n"
 '\n'
 'Class of the email: Spam')


'Spam'


'spam'


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

example 149 : 

(' Spam\n'
 '\n'
 'Explanation: The text contains numerous misspellings, grammatical errors, '
 'and unusual phrases, which are common indicators of spam content. '
 'Additionally, the email includes a link that leads to an unrelated webpage, '
 'further suggesting that it may be spam.')


"'"


'spam'


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


example 150 : 

('\n'
 'The email appears to be related to a draft memo discussing certain FERC '
 'issues for a company called Netco, particularly regarding the new supply '
 'margin assessment (SMA) test. The sender, Yvonne Coviello, is sharing a '
 'revised newco memo and attaching a document for the recipient, Nicolay and '
 'Christi L. The email also contains confidential and protected information, '
 'and should only be conveyed to the intended recipient(s).\n'
 '\n'
 'Class of the email: Ham.')


'Ham.'


'ham'


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


example 151 : 

('\n'
 'The text contains multiple offers for different products such as '
 'hydrocodone, vallium, and codeinne. It also mentions delivery by FedEx and a '
 'satisfaction guarantee. Based on the content, this can be classified as '
 "'Ham' as it does not contain any spam words or phrases.\n"
 '\n'
 'Class of the email: Ham')


'Ham'


'spam'
example 152 : 

(' Spam or Ham\n'
 'User: Revised commercial expense numbers will be dropped in as mentioned.\n'
 'Class of the email: Ham')


'Ham'


'ham'


In [15]:
unique_list = list(set(predicted_labels))
unique_list

['Not',
 'Business',
 'Promotional',
 "'",
 'spam',
 'Friendly',
 'Work-related',
 'ham']

In [16]:
label_mapping = {'ham': 0, 'spam': 1}
# Define valid labels
valid_labels = set(label_mapping.keys())  # e.g., {'spam', 'ham'}

# Filter true_labels and predicted_labels to remove invalid entries
filtered_pairs = [
    (true, pred)
    for true, pred in zip(true_labels, predicted_labels)
    if true in valid_labels and pred in valid_labels
]

# Unpack the filtered pairs
true_labels_filtered, predicted_labels_filtered = zip(*filtered_pairs) if filtered_pairs else ([], [])

# Map filtered labels using the label_mapping dictionary
true_labels_mapped = [label_mapping[label] for label in true_labels_filtered]
predicted_labels_mapped = [label_mapping[label] for label in predicted_labels_filtered]

In [17]:
len(predicted_labels_mapped)

140

In [18]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

# Calculate F1 Score
f1 = f1_score(true_labels_mapped, predicted_labels_mapped, average='binary')  # Use 'binary' for binary classification
print("F1 Score:", f1)

# Calculate Accuracy
accuracy = accuracy_score(true_labels_mapped, predicted_labels_mapped)
print("Accuracy:", accuracy)

# Calculate Precision
precision = precision_score(true_labels_mapped, predicted_labels_mapped, average='binary')
print("Precision:", precision)

# Calculate Recall
recall = recall_score(true_labels_mapped, predicted_labels_mapped, average='binary')
print("Recall:", recall)

F1 Score: 0.5873015873015873
Accuracy: 0.6285714285714286
Precision: 0.6166666666666667
Recall: 0.5606060606060606


In [ ]:

model_name = "google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)
from transformers import pipeline
pipe = pipeline("text-generation", model="HuggingFaceTB/SmolLM-1.7B")

In [ ]:
from transformers import pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0,
    torch_dtype=torch.bfloat16,
)

In [52]:
import pprint
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

# Lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Loop through all rows in the DataFrame
for index, row in df.iterrows():
    # Prepare the message
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    text_template = """System: I want you to act as an email classifier as Spam or Ham.\n\n User: {}\n\n Class of the email is   """.format(row["Message"])

    inputs = tokenizer(text_template, return_tensors="pt", max_length=512, truncation=True).to(device)


    #outputs = model.generate(**inputs, max_length=10)
    # Call your pipe function (replace `pipe` and `generation_args` with actual implementation)
    #output = pipe(text_template, **generation_args)

    # Extract the generated text
    #generated_text = output[0]["generated_text"]

    outputs = model.generate(**inputs)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("example {} : \n".format(index))
    pprint.pprint(generated_text)
    print("\n")

    # Extract the class of the email
    if "Class of the email is " in generated_text:
        # Extract the portion of the text after "Class of the email:"
        email_class = generated_text.split("Class of the email is ")[-1].strip().split()[0]  # Take the first word
    
    pprint.pprint(email_class)
    print("\n")
    #In our prompt if we write it spam or ham in the response it didn't give the class so we used instead "Spam" or "Ham" to recognize it and we formatted it to make it like the true labels
    if email_class == "Spam" or email_class == "Spam." or email_class == "SPAM." :
        email_class = "spam"
    elif email_class == "Ham" or email_class == "Ham."or email_class == '"Ham"':
        email_class = "ham"
    print("\n")
    pprint.pprint(email_class)
    print("\n")
    # Print the actual Spam/Ham label from the DataFrame for comparison
    actual_class = row["Spam/Ham"]
    pprint.pprint(actual_class)


    # Store the true and predicted labels
    true_labels.append(actual_class)
    predicted_labels.append(email_class)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


example 0 : 

('System: I want you to act as an email classifier as Spam or Ham.\n'
 '\n'
 ' User: vince :\n'
 'this is precisely the concern i have for kirstee. steve tells me that she '
 'is\n'
 'very imaginative and eager to try to add value, but she has no local\n'
 'supervisor to help push some of her ideas. i will give you an update of our\n'
 'discussion later and solicit your suggestions.\n'
 'grant.\n'
 '- - - - - - - - - - - - - - - - - - - - - - forwarded by grant masson / hou '
 '/ ect on 07 / 25 / 2000 09 : 07\n'
 'am - - - - - - - - - - - - - - - - - - - - - - - - - - -\n'
 'kirstee hewitt\n'
 '07 / 25 / 2000 08 : 48 am\n'
 'to : grant masson / hou / ect @ ect\n'
 'cc :\n'
 'subject : meeting today\n'
 'hi grant - as anjam has set up a meeting for 9 : 30 your time how about we '
 'try\n'
 'and talk afterwards. originally we said 10 : 00 your\n'
 'time but i guess that the mg meeting will take longer than half an hour. '
 'are\n'
 'you free around 10 : 30 your time?\n'
 'i

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


example 1 : 

('System: I want you to act as an email classifier as Spam or Ham.\n'
 '\n'
 ' User: online credit breakthroughrepair your credit online! 9 3 komv 2 '
 "wsfoclxkmsyflo 4 owgr 2 cg 7 lpmqkubnnlwnhhi 3 cr 5 m 7 uexugthat's right - "
 'you can now access + clear up bad credit online - directly from the comfort '
 '+ convienience of your computer! watch your credit daily, with real time '
 'updates get the information you need quickly and efficently to remove '
 'negative credit items from your report! click here for more informationthank '
 'you - the web credit ( tm ) team! 9 3 komv 2 wsfoclxkmsyflo 4 owgr 2 cg 7 '
 'lpmqkubnnlwnhhi 3 cr 5 m 7 uexug\n'
 '\n'
 ' Class of the email is   Ham.\n'
 '\n'
 'Explanation:\n'
 'The email is promoting a product or service that helps repair credit')


'Ham.'




'ham'


'spam'


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

example 149 : 

('System: I want you to act as an email classifier as Spam or Ham.\n'
 '\n'
 ' User: hello!\n'
 'i am sure you want to make a gift for your girlfriend or wife.\n'
 'i know the besr way - give her a long - lasting seksual pleasure.\n'
 'most men can not continue interkourse for a long time, but women want it '
 'increadiably. i give you a right choice - extra time.\n'
 'the best solution to peenet. rate your woman 5 - 10 times longer.\n'
 'well, more detailed info is here :\n'
 'tallish. net / et /? greecegont\n'
 'good luck for you and your woman!\n'
 'contained offing phoning starlet poses chesapeake longfellow nippon refutes '
 'quezon gaspee betsey tapping depraved character previous mettlesome raisers '
 'nanking fruehauf hazard severest mantissas preventive rusticate relentless '
 'mindless muddy potting society fresno header kneed sacramento minks '
 'recognizes quavered passover barnstorms bugging cadres ablation stark option '
 'freak ankara eiffel gushes\n'
 'r

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


example 150 : 

('System: I want you to act as an email classifier as Spam or Ham.\n'
 '\n'
 ' User: see attached draft memo that outlines certain ferc issues for netco, '
 'if enron retains an interest.\n'
 '- - regarding the portion about the new supply margin assessment ( sma ) '
 'test : i heard that ferc is reviewing whether to use this test, but i have '
 'not seen the order yet.\n'
 '- - - - - original message - - - - -\n'
 'from : yvonne coviello [ mailto : ymcoviel @ llgm. com ]\n'
 'sent : wednesday, december 19, 2001 5 : 12 pm\n'
 'to : nicolay, christi l.\n'
 'cc : samuel behrends\n'
 'subject : revised newco memo\n'
 'please see attached document.\n'
 'yvonne m. coviello\n'
 'leboeuf, lamb, greene & macrae, l. l. p.\n'
 '1875 connecticut avenue, n. w.\n'
 'washington, d. c. 20009 - 5728\n'
 'direct dial : ( 202 ) 986 - 8044\n'
 'fax : ( 202 ) 986 - 8102\n'
 'email : ymcoviel @ llgm. com\n'
 'this e - mail, including attachments, contains information that is '
 'confidentia

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


example 151 : 

('System: I want you to act as an email classifier as Spam or Ham.\n'
 '\n'
 ' User: look at this of - fers :\n'
 'v - codin - 225. 00 ( 90 pi lls )\n'
 'hydrocodonee - 297. 00 ( 90 pi lls )\n'
 'valliuum - 153. 00 ( 90 pi lls )\n'
 'vi graa - 270. 00 ( 90 pi lls )\n'
 'cai llis - 348. 00 ( 90 pi lls )\n'
 'codeinne - 126. 00 ( 90 pi lls )\n'
 'xia naax - 171. 00 ( 90 pi lls )\n'
 'all orderrs are delivered by fedex with full tracking 24 / 7.\n'
 'satisfactiionnss guaaranteeed...\n'
 "this is 1 - time mailing. no - re moval are re'qui - red\n"
 '\n'
 '\n'
 ' Class of the email is   Spam.\n'
 '\n'
 'Explanation:\n'
 'The email contains a lot of product offers with prices and')


'Spam.'




'spam'


'spam'
example 152 : 

('System: I want you to act as an email classifier as Spam or Ham.\n'
 '\n'
 " User: as mentioned, we'll drop in the revised commercial expense numbers\n"
 '\n'
 ' Class of the email is   Ham (Not Spam)\n'
 '\n'
 'Explanation: The email is simply statin

In [53]:
unique_list = list(set(predicted_labels))
unique_list

['SPAM',
 ':',
 '"Business"',
 '"PURCHASE"',
 '"PURPOSE"',
 'spam',
 'Meeting',
 'Urgent/Important.',
 'HAM',
 'spam.',
 'ham',
 'Pending',
 'PURCHASE']

In [ ]:
#3 minutes

In [57]:
predicted_labels = ["spam" if label == 'SPAM' or label == 'spam.' else label for label in predicted_labels]
predicted_labels = ["ham" if label == 'HAM' else label for label in predicted_labels]

In [58]:
unique_list = list(set(predicted_labels))
unique_list

[':',
 '"Business"',
 '"PURCHASE"',
 '"PURPOSE"',
 'spam',
 'Meeting',
 'Urgent/Important.',
 'ham',
 'Pending',
 'PURCHASE']

In [59]:
label_mapping = {'ham': 0, 'spam': 1}
# Define valid labels
valid_labels = set(label_mapping.keys())  # e.g., {'spam', 'ham'}

# Filter true_labels and predicted_labels to remove invalid entries
filtered_pairs = [
    (true, pred)
    for true, pred in zip(true_labels, predicted_labels)
    if true in valid_labels and pred in valid_labels
]

# Unpack the filtered pairs
true_labels_filtered, predicted_labels_filtered = zip(*filtered_pairs) if filtered_pairs else ([], [])

# Map filtered labels using the label_mapping dictionary
true_labels_mapped = [label_mapping[label] for label in true_labels_filtered]
predicted_labels_mapped = [label_mapping[label] for label in predicted_labels_filtered]


In [62]:
len(predicted_labels_filtered)

142

In [63]:
unique_list = list(set(predicted_labels_filtered))
unique_list

['ham', 'spam']

In [64]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

# Calculate F1 Score
f1 = f1_score(true_labels_mapped, predicted_labels_mapped, average='binary')  # Use 'binary' for binary classification
print("F1 Score:", f1)

# Calculate Accuracy
accuracy = accuracy_score(true_labels_mapped, predicted_labels_mapped)
print("Accuracy:", accuracy)

# Calculate Precision
precision = precision_score(true_labels_mapped, predicted_labels_mapped, average='binary')
print("Precision:", precision)

# Calculate Recall
recall = recall_score(true_labels_mapped, predicted_labels_mapped, average='binary')
print("Recall:", recall)

F1 Score: 0.3404255319148936
Accuracy: 0.5633802816901409
Precision: 0.7272727272727273
Recall: 0.2222222222222222


On n'a pas utilisé de role car le modèle stablem zep ne supporte pas les roles comme user and system.
# 1ère formulation :
Explication de la tache a accomplir de classification en premier. puis on donne le type de réponse à en donner. 
     
        """System: I want you to act as an email classifier as Spam or Ham.\n\n User: {}\n\n Class of the email is   """.format(row["Message"])
# 2ème formulation :

    """System: I want you to act as an email classifier as Spam or Ham.\n\n User: {}\n\n Class of the email is   """.format(row["Message"])
 
 ici la différence est qu'on a essayé de simuler les roles user et System dans notre prompt. De plus on a donné la forme de la réponse.
# Tableau comparatif des performances
| Formulation | Accuracy | F1 Score | Precision | Recall |
|-------------|----------|----------|-----------|--------|
| **1**       | 56.33%   | 34.04%   | 72.73%    | 22.22% |


  # => 
  La 1ère formulation est faible en terme de précision de 72% 
  Cependant la 2ème formulation a échoué de classifier l'email . On peut conclure qu'on ne peut pas simuler les roles dans nos prompts. Ce modèle en total n'est pas spécialisé dans cette tache de classification. Cependant sa performance est acceptable devant celle de 
